In [23]:
# Basic concepts 
# Word embeddings are dense vectors of real numbers, one per word in your vocabulary
# inputs are |V| dimension where V is vocabulary 
# outpust are a few dimension vector 

# we need to find semantic similarity between sentences 
# fundamental linguistic assumption: distributional hypothesis
# words appearing in similar contexts are related to each other semantically

# 'semantic attribute' such as "can run", "likes coffee", "majored in Physics" 
# give each word a vector [2.3,9.4,...], each feature is a 'semantic attribute'
# as a consequence, we can calculate "semantic similarity" by cos(theta) between two vector 
# let word embeddings as parameters of model to be learnt, they are perhaps unterpretable

# word embedding in Pytorch
# embeddings are stored as |V|xD matrix, where row represents the index of a word 

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1) # set seeds for generating random numbers 

word_to_ix = {"hello": 0, "world": 1} # dirctionary of vocabulary
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.LongTensor([word_to_ix["hello"]]) # index of "hello" in tensor  
hello_embed = embeds(autograd.Variable(lookup_tensor))
print(hello_embed) 


Variable containing:
-2.9718  1.7070 -0.4305 -2.2820  0.5237
[torch.FloatTensor of size 1x5]



In [24]:
# N-Gram Language Modeling 
# for a sequence of words w, we calculate w_i from w_i-n+1 to w_i
CONTEXT_SIZE = 2 # n = 2
EMBEDDING_DIM = 10 # embedding_dim

test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)

trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
print(trigrams[:3]) # print the first 3 trigrams
vocab = set(test_sentence) # vocabulary of sentence 
print(vocab) 
word_to_ix = {word: i for i, word in enumerate(vocab)}
print(word_to_ix) # word and refered index 

class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out) # log-softmax for output
        return log_probs        

losses = []
loss_function = nn.NLLLoss() # Negative log loss

model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)



[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]
{'thy', 'succession', 'now,', 'to', 'worth', 'days;', 'How', 'besiege', 'being', 'winters', 'brow,', 'much', 'When', 'be', 'cold.', 'within', 'answer', 'in', 'thou', 'new', 'made', 'dig', 'so', 'Were', 'Proving', 'say,', 'fair', 'weed', 'mine', 'gazed', 'asked,', 'couldst', "youth's", 'small', 'thriftless', 'of', 'a', 'it', 'all-eating', 'own', 'sunken', 'and', 'blood', 'And', 'art', 'old,', 'To', 'see', 'eyes,', "beauty's", 'beauty', 'livery', 'treasure', 'lusty', 'Thy', 'Will', 'make', 'Shall', "totter'd", 'old', 'praise', 'child', 'count,', 'lies,', 'praise.', "excuse,'", 'proud', 'thine!', 'deep', 'where', 'If', 'use,', 'on', 'This', "feel'st", 'all', 'the', 'trenches', 'shall', 'thine', "'This", 'an', 'warm', 'more', 'his', 'by', 'held:', 'sum', 'when', 'my', 'Then', "deserv'd", 'were', 'field,', 'Where', 'shame,', 'forty'}
{'thy': 0, 'succession': 1, 'now,': 2, 'to': 3, 'worth': 4

In [25]:
for epoch in range(10):
    total_loss = torch.Tensor([0])
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in variables)
        context_idxs = [word_to_ix[w] for w in context]
        context_var = autograd.Variable(torch.LongTensor(context_idxs))

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_var)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a variable)
        loss = loss_function(log_probs, autograd.Variable(
            torch.LongTensor([word_to_ix[target]])))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        total_loss += loss.data
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[
 520.2759
[torch.FloatTensor of size 1]
, 
 517.9081
[torch.FloatTensor of size 1]
, 
 515.5548
[torch.FloatTensor of size 1]
, 
 513.2136
[torch.FloatTensor of size 1]
, 
 510.8837
[torch.FloatTensor of size 1]
, 
 508.5640
[torch.FloatTensor of size 1]
, 
 506.2554
[torch.FloatTensor of size 1]
, 
 503.9563
[torch.FloatTensor of size 1]
, 
 501.6659
[torch.FloatTensor of size 1]
, 
 499.3838
[torch.FloatTensor of size 1]
]


In [26]:
# predict words meaning with continous-bag-of-words

CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right, length of bag-of-words
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)
word_to_ix = {word: i for i, word in enumerate(vocab)}

data = []
for i in range(2, len(raw_text) - 2):
    # before and after context
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i] # target word to be predicted
    data.append((context, target))
print(data[:5])

class CBOW(nn.Module):

    def __init__(self):
        pass

    def forward(self, inputs):
        pass

# create your model and train.  here are some functions to help you make
# the data ready for use by your module


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)


make_context_vector(data[0][0], word_to_ix)  # example  

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


Variable containing:
 21
 36
  1
 13
[torch.LongTensor of size 4]

In [27]:
print(data)

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea'), (['the', 'idea', 'a', 'computational'], 'of'), (['idea', 'of', 'computational', 'process.'], 'a'), (['of', 'a', 'process.', 'Computational'], 'computational'), (['a', 'computational', 'Computational', 'processes'], 'process.'), (['computational', 'process.', 'processes', 'are'], 'Computational'), (['process.', 'Computational', 'are', 'abstract'], 'processes'), (['Computational', 'processes', 'abstract', 'beings'], 'are'), (['processes', 'are', 'beings', 'that'], 'abstract'), (['are', 'abstract', 'that', 'inhabit'], 'beings'), (['abstract', 'beings', 'inhabit', 'computers.'], 'that'), (['beings', 'that', 'computers.', 'As'], 'inhabit'), (['that', 'inhabit', 'As', 'they'], 'computers.'), (['inhabit', 'computers.', 'they', 'evolve,'], 'As'), (['computers.', 'As', 'evolve,', 'processes']